In [3]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [4]:
a = open('rap_west.txt')
rappers_west = a.read().split('\n')

b = open('rap_east.txt')
rappers_east = b.read().split('\n')

In [5]:
base_url = 'https://api.spotify.com/v1/'

client_id = '6856620ebab6449e84df2b92bf144f72'
client_secret = '703e965c291b41169bc00ff4e35fc211'

In [6]:
### getting the access token from Spotify

auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(auth_url, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [7]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

params = {
    'limit':1
}

*WEST COAST:*

In [8]:
### collecting artist ids from Spotify API:

west_artist_ids = {}
unaligned = {} # collects artist names/ids where the search term is different to the Spotify artist name
errors_west = [] # errors - to try again

for artist in tqdm(rappers_west):
    
    # putting the search term into the correct format for the request (e.g. 'Kid Frost' -> 'Kid%20Frost')
    search_term = artist.replace(' ', '%20')
    
    # constructing the query for the request, and pushing the get request, converting into json.
    query_string = base_url + 'search?' + f'q={search_term}&type=artist'
    r = requests.get(query_string, headers=headers, params=params)
    r = r.json()
    
    try:
        # getting the artist id from within the json material
        west_artist_ids[artist] = r['artists']['items'][0]['id']
        
        # getting the Spotify artist name associated with the id to check that we are finding the right artist
        spotify_name = r['artists']['items'][0]['name'].lower()
        if spotify_name != artist.lower():
            unaligned[artist] = spotify_name
        else:
            continue
    except:
        errors_west.append(artist)

100%|██████████| 388/388 [00:53<00:00,  7.27it/s]


In [9]:
len(west_artist_ids), len(errors_west)

(380, 8)

In [10]:
### dealing with the errors:

for artist in tqdm(errors_west):
    
    search_term = artist.replace(' ', '%20')
    
    query_string = base_url + 'search?' + f'q={search_term}&type=artist'
    r = requests.get(query_string, headers=headers, params=params)
    r = r.json()
    
    try:
        west_artist_ids[artist] = r['artists']['items'][0]['id']
        
        spotify_name = r['artists']['items'][0]['name'].lower()
        if spotify_name != artist.lower():
            unaligned[artist] = spotify_name
        else:
            continue
        errors_west.remove(artist)
    except:
        continue

100%|██████████| 8/8 [00:00<00:00,  8.83it/s]


In [11]:
len(west_artist_ids), len(errors_west)

(380, 8)

In [12]:
len(set(list(west_artist_ids.keys())))

380

In [13]:
# aligned
# tupac - correct.
# n.w.a. - correct
# paris - wrong.
# drop the rest.

In [14]:
### dropping invalid entries (inspected the unaligned list, most were not relevant to the project):
for item in list(west_artist_ids):
    
    if len(item) == 1:
        west_artist_ids.pop(str(item))
    
    elif item in list(unaligned.keys()):
        if item not in ['Tupac Shakur', 'Paris', 'N.W.A']:
            west_artist_ids.pop(str(item))

In [15]:
### checking the unaligned dictionary, spotify returned a different artist to the desired rapper.
### By investigating another artist search request, we can see that this rapper is 9th on the list of artists.
artist = 'Paris'
search_term = artist.replace(' ', '%20')
query_string = base_url + 'search?' + f'q={search_term}&type=artist'
r = requests.get(query_string, headers=headers)
r = r.json()
west_artist_ids[artist] = r['artists']['items'][9]['id']

In [16]:
len(west_artist_ids)

301

In [17]:
### filtering remaining artists on genre (lots of artists irrelevant to the problem):


filter_errors_west = {}
popped = {}

for artist in tqdm(list(west_artist_ids.keys())):
    
    artist_id = west_artist_ids[artist]

    # artist request using the artist ids - returns among other things the genre that the rapper falls within
    r = requests.get(base_url + 'artists/' + artist_id, 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()

    try:
        # filtering on genres
        if ('hip hop' not in ' '.join(d['genres'])) and ('rap' not in ' '.join(d['genres'])):

            east_artist_ids.pop(str(artist))
            popped[artist] = artist_id
        
        # some genres to avoid, found by investigating the artists' spotify info
        banned = ['chicano rap', 'rap rock', 'memphis hip hop', 'soul flow', 'gospel rap', 'christian trap',
                 'psychedelic hip hop', 'dirty texas rap', 'southern hip hop', 'trap queen']
        for word in banned:    
            if word in ' '.join(d['genres']): ### new part. Needed for sure. Also: trap queen.

                west_artist_ids.pop(str(artist))
                popped[artist] = artist_id
            
        else:
            print(d['genres'])

    except:
        print('here')
        filter_errors_west[artist] = artist_id

  1%|          | 2/301 [00:00<00:27, 10.76it/s]

here
here
here


  2%|▏         | 5/301 [00:00<00:27, 10.62it/s]

['abstract hip hop']
['gangster rap', 'west coast rap']
['g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap']


  3%|▎         | 9/301 [00:00<00:30,  9.59it/s]

...

here
here
here


 99%|█████████▉| 298/301 [00:31<00:00, 11.58it/s]

here
here
['hyphy', 'oakland hip hop', 'west coast rap', 'west coast trap']


100%|██████████| 301/301 [00:31<00:00,  9.53it/s]

['chicago rap', 'pop rap']
['abstract hip hop', 'alternative hip hop', 'hip hop', 'underground rap']


In [18]:
len(west_artist_ids)

236

In [19]:
### filtering error handling: errors --- there were entries for which the artist id request didn't work...

# upped: those errors which are added to the list of artists
upped = {}

for artist in tqdm(list(filter_errors_west.keys())):
    
    artist_id = filter_errors_west[artist]

    r = requests.get(base_url + 'artists/' + artist_id, 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()

    try:
        # same criteria for genres as earlier.
        if ('hip hop' in ' '.join(d['genres'])) or ('rap' in ' '.join(d['genres'])):

            west_artist_ids[artist] = artist_id
            filter_errors_west.pop(artist)
            upped[artist] = artist_id
            
        else:
            print(d['genres'])

    except:
        print('here')
        filter_errors_west[artist] = artist_id

  3%|▎         | 2/74 [00:00<00:06, 11.46it/s]

[]
['g funk', 'hyphy']
[]


  8%|▊         | 6/74 [00:00<00:05, 11.88it/s]

[]
['g funk']
[]


 11%|█         | 8/74 [00:00<00:05, 12.09it/s]

...

[]
['hyphy']
[]


 97%|█████████▋| 72/74 [00:06<00:00, 12.02it/s]

[]
[]


100%|██████████| 74/74 [00:08<00:00,  8.82it/s]

[]


In [20]:
len(west_artist_ids)

237

In [21]:
### filtering on popularity:
filter_errors_west = {}
popped = {}

for artist in tqdm(list(west_artist_ids.keys())):
    
    artist_id = west_artist_ids[artist]

    r = requests.get(base_url + 'artists/' + artist_id, 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()

    try:
        # spotify has a popularity scale between 0 and 100
        if d['popularity'] < 55:
            
            west_artist_ids.pop(str(artist))
            popped[artist] = artist_id
            
        else:
            print(d['popularity'])

    except:
        print('here')
        filter_errors_west[artist] = artist_id

  3%|▎         | 8/237 [00:02<01:33,  2.46it/s]

81


  5%|▍         | 11/237 [00:02<00:56,  3.99it/s]

77


  8%|▊         | 19/237 [00:03<00:28,  7.63it/s]

...

67
59


 97%|█████████▋| 229/237 [00:22<00:00, 11.75it/s]

67


100%|██████████| 237/237 [00:23<00:00, 10.28it/s]

55
57


In [22]:
west_artist_ids.pop('Tupac Shakur')
# there was a duplicate

'1ZwdS5xdxEREPySFridCfh'

In [23]:
with open('west_artist_ids_HUGE.json', 'w') as fp:
    json.dump(west_artist_ids, fp)

*EAST COAST:*

*Code is repeated for east coast*

In [24]:
### collecting artist ids:

east_artist_ids = {}
aligned = {}
errors_east = []

for artist in tqdm(rappers_east):
    
    search_term = artist.replace(' ', '%20')
    
    query_string = base_url + 'search?' + f'q={search_term}&type=artist'
    r = requests.get(query_string, headers=headers, params=params)
    r = r.json()
    
    try:
        east_artist_ids[artist] = r['artists']['items'][0]['id']
        
        spotify_name = r['artists']['items'][0]['name'].lower()
        if spotify_name != artist.lower():
            aligned[artist] = spotify_name
        else:
            continue
    except:
        errors_east.append(artist)

100%|██████████| 460/460 [00:49<00:00,  9.27it/s]


In [25]:
### dealing with errors:

for artist in tqdm(errors_east):
    
    search_term = artist.replace(' ', '%20')
    
    query_string = base_url + 'search?' + f'q={search_term}&type=artist'
    r = requests.get(query_string, headers=headers, params=params)
    r = r.json()
    
    try:
        east_artist_ids[artist] = r['artists']['items'][0]['id']
        
        spotify_name = r['artists']['items'][0]['name'].lower()
        if spotify_name != artist.lower():
            aligned[artist] = spotify_name
        else:
            continue
        errors_east.remove(artist)
    except:
        continue

100%|██████████| 70/70 [00:06<00:00, 11.04it/s]


In [26]:
for item in list(east_artist_ids):
    
    if len(item) == 1:
        east_artist_ids.pop(str(item))
        
    elif item in list(aligned.keys()):
        if item not in []:
            east_artist_ids.pop(str(item))

In [27]:
### rappers where spotify has returned a different artist

artist = 'Eric B. & Rakim'
search_term = artist.replace(' ', '%20')
query_string = base_url + 'search?' + f'q={search_term}&type=artist'
r = requests.get(query_string, headers=headers)
r = r.json()
east_artist_ids[artist] = r['artists']['items'][1]['id']

artist = 'AZ'
search_term = artist.replace(' ', '%20')
query_string = base_url + 'search?' + f'q={search_term}&type=artist'
r = requests.get(query_string, headers=headers)
r = r.json()
east_artist_ids[artist] = r['artists']['items'][2]['id']

artist = 'Lauryn Hill'
search_term = artist.replace(' ', '%20')
query_string = base_url + 'search?' + f'q={search_term}&type=artist'
r = requests.get(query_string, headers=headers)
r = r.json()
east_artist_ids[artist] = r['artists']['items'][0]['id']

In [28]:
### filtering on genre:
filter_errors_east = {}
popped = {}

for artist in tqdm(list(east_artist_ids.keys())):
    
    artist_id = east_artist_ids[artist]

    r = requests.get(base_url + 'artists/' + artist_id, 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()

    try:
        if ('hip hop' not in ' '.join(d['genres'])) and ('rap' not in ' '.join(d['genres'])):

            east_artist_ids.pop(str(artist))
            popped[artist] = artist_id
            
        banned = ['dark trap', 'buffalo hip hop', 'boston hip hop', 'trap queen', 'christian trap', 
                  'emo rap', 'vapor trap']
        for word in banned:    
            if word in ' '.join(d['genres']):

                east_artist_ids.pop(str(artist))
                popped[artist] = artist_id
            
        else:
            print(d['genres'])

    except:
        print('here')
        filter_errors_east[artist] = artist_id

  1%|          | 3/307 [00:00<00:27, 10.96it/s]

['emo rap']
['wu fam']
['brooklyn drill', 'nyc rap']


  2%|▏         | 5/307 [00:00<00:26, 11.23it/s]

['alternative hip hop', 'boom bap', 'underground hip hop']
[]
['east coast hip hop', 'gangster rap', 'hip hop', 'pop rap', 'queens hip hop', 'rap']


  3%|▎         | 9/307 [00:00<00:27, 11.00it/s]

...

[]
['jewish hip hop']
['dark trap']


100%|█████████▉| 306/307 [00:29<00:00, 12.39it/s]

['deep underground hip hop']
['alternative hip hop', 'bboy', 'east coast hip hop', 'electro', 'gangster rap', 'hardcore hip hop', 'hip hop', 'old school hip hop', 'rap']
['underground hip hop']


100%|██████████| 307/307 [00:29<00:00, 10.28it/s]

['conscious hip hop', 'dance pop', 'hip pop', 'neo soul', 'new jersey rap', 'r&b', 'urban contemporary']


In [29]:
### filtering on popularity:
filter_errors_east = {}
popped = {}

for artist in tqdm(list(east_artist_ids.keys())):
    
    artist_id = east_artist_ids[artist]

    r = requests.get(base_url + 'artists/' + artist_id, 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()

    try:
        if d['popularity'] < 55:
            
            east_artist_ids.pop(str(artist))
            popped[artist] = artist_id
            
        else:
            print(d['popularity'])

    except:
        print('here')
        filter_errors_east[artist] = artist_id

  0%|          | 0/189 [00:00<?, ?it/s]

63


  2%|▏         | 3/189 [00:00<00:17, 10.83it/s]

84


  2%|▏         | 4/189 [00:00<00:17, 10.37it/s]

...

here
here


 98%|█████████▊| 185/189 [00:16<00:00, 11.42it/s]

69


100%|██████████| 189/189 [00:16<00:00, 11.48it/s]

55
58
70


In [41]:
east_artist_ids

{'22Gz': '4JhbRL6zaItAyzqx4gHTqz',
 '50 Cent': '3q7HBObVc0L8jNeTe5Gofh',
 '070 Shake': '12Zk1DFhCbHY6v3xep2ZjI',
 'Sammy Adams': '1S9WGrK7SZfXXQtHBYzDle',
 'Aesop Rock': '2fSaE6BXtQy0x7R7v9IOmZ',
 'Hoodie Allen': '382aq8Pij5V2nE2JMHMoxl',
 'Lloyd Banks': '3vDUJHQtqT3jFRZ2ECXDTi',
 'Big Pun': '2Xu7q46Hf02xOoEIm4E1Qs',
 'Black Thought': '6DJEUXZm0e2rAohdoZ5Voo',
 'A Boogie wit da Hoodie': '31W5EY0aAly4Qieq6OFu6I',
 'Action Bronson': '7BMccF0hQFBpP6417k1OtQ',
 'Cappadonna': '0dS5xeDefwoUNasLUHnCjR',
 'Cardi B': '4kYSro6naA4h99UJvo89HB',
 'Desiigner': '7pFeBzX627ff0VnN6bxPR4',
 'DJ Drama': '5oNgAs7j5XcBMzWv3HAnHG',
 'Dave East': '7e10JUMF7MJmmwYpnTSMI5',
 'Fabolous': '0YWxKQj2Go9CGHCp77UOyy',
 'Fat Joe': '3ScY9CQxNLQei8Umvpx5g6',
 'Felly': '2848adRcxvgWNRcz1g1tQD',
 'Fivio Foreign': '14CHVeJGrR5xgUGQFV5BVM',
 'Ghostface Killah': '6FD0unjzGQhX3b6eMccMJe',
 'Godfather Don': '7j91LXmuyKBXTykkvYfrQ2',
 'Jean Grae': '2ipBhKpOYqs6BbysLNGye6',
 'Cory Gunz': '7BAaznGSsqcksud4R5ou5z',
 'GZA': '6ns6

In [44]:
with open('east_artist_ids_HUGE.json', 'w') as fp:
    json.dump(east_artist_ids, fp)

Using Spotify API to find all songs for the rappers in each list which were released between 1987-2000 (inclusive):

In [46]:
full_song_dict_east = {}

for artist in tqdm(east_artist_ids):
    
    artist_id = east_artist_ids[artist]
    
    r = requests.get(base_url + 'artists/' + artist_id + '/albums', 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()
    
    artist_data = {}   # to hold track info within each iteration of the loop
    albums = [] # to avoid duplicate albums

    # get tracks from all albums for each artist
    for album in d['items']:
        album_name = album['name']

        # skip the album if we've already got it
        trim_name = album_name.split('(')[0].strip() # stuff like '(Deluxe Edition)'
        if trim_name.lower() in albums or int(album['release_date'][:4])>1999 or int(album['release_date'][:4]) < 1987:
            continue
        albums.append(trim_name.lower())

        # get all tracks from album
        r = requests.get(base_url + 'albums/' + album['id'] + '/tracks', 
            headers=headers)
        tracks = r.json()['items']

        track_names = []
        for track in tracks:

            track_name = track['name'].split(' - ')[0]

            if track_name not in track_names: # avoid duplicates
                track_names.append(track_name)

        # add to tracks to artist data dictionary, with the album name as key
        artist_data[trim_name] = track_names
        
    # add these tracks to the full song dictionary
    full_song_dict_east[artist] = artist_data
        

100%|██████████| 91/91 [00:35<00:00,  2.55it/s]


In [47]:
## dropping the artists for which there are no tracks entered (because they were released outside of the window)

for artist in list(full_song_dict_east.keys()):
    
    if full_song_dict_east[artist] == {}:
        
        full_song_dict_east.pop(artist)
        
len(full_song_dict_east)

30

In [50]:
### manually checking the albums for each artist - dropping the undetected duplicates

full_song_dict_east[rappers_east[43]]
# indices here are from rappers_east.
# wu-tang clan (1)... only 2 albums in this period, drop the weird first one...
# nas (2) --- double illmatic. also drop 'iTunes Originals'
# (5) & (7) - both contain 'blackout'
# (7) - double tical 2000.
# (7) - only keep tical. Sorts this 5/7 mess.
# (9) - double 'paid in full'
# (12) - double 'the 18th letter'
# (14) - double n**** please
# (16) - japanese characters album... And double 'AKA the rugged child'
# (40) - drop 'the most beautifullest hits'
# (45) - 3 straight out the jungles

Same for West:

In [31]:
full_song_dict_west = {}

for artist in tqdm(west_artist_ids):
    
    artist_id = west_artist_ids[artist]
    
    r = requests.get(base_url + 'artists/' + artist_id + '/albums', 
                     headers=headers, 
                     params={'include_groups': 'album', 'limit': 50})
    d = r.json()
    
    artist_data = {}
    albums = []

    for album in d['items']:
        album_name = album['name']

        trim_name = album_name.split('(')[0].strip()
        if trim_name.upper() in albums or int(album['release_date'][:4])>1999 or int(album['release_date'][:4])<1987:
            continue
        albums.append(trim_name.upper())

        print(album_name)

        r = requests.get(base_url + 'albums/' + album['id'] + '/tracks', 
            headers=headers)
        tracks = r.json()['items']

        track_names = []
        for track in tracks:

            track_name = track['name'].split(' - ')[0]

            if track_name not in track_names:
                track_names.append(track_name)

        artist_data[trim_name] = track_names
        
    
    full_song_dict_west[artist] = artist_data
        

  0%|          | 0/55 [00:00<?, ?it/s]

R U Still Down? [Remember Me]
All Eyez On Me
Me Against The World


  2%|▏         | 1/55 [00:00<00:29,  1.84it/s]

Strictly 4 My N.I.G.G.A.Z...
2Pacalypse Now


  9%|▉         | 5/55 [00:01<00:16,  3.02it/s]

...

 95%|█████████▍| 52/55 [00:12<00:00,  3.58it/s]

Bow Down
40 Dayz & 40 Nightz (Explicit)
At The Speed Of Life


 96%|█████████▋| 53/55 [00:12<00:00,  2.59it/s]

The Recipe
Based On A True Story
Mack 10


100%|██████████| 55/55 [00:13<00:00,  4.17it/s]


In [32]:
for artist in list(full_song_dict_west.keys()):
    
    if full_song_dict_west[artist] == {}:
        
        full_song_dict_west.pop(artist)
        
len(full_song_dict_west)

26

In [40]:
full_song_dict_west

{'2Pac': {'R U Still Down? [Remember Me]': ['Redemption',
   'Open Fire',
   'R U Still Down? (Remember Me)',
   'Hellrazor',
   'Thug Style',
   'Where Do We Go From Here (Interlude)',
   'I Wonder If Heaven Got A Ghetto',
   'Nothing To Lose',
   "I'm Gettin Money",
   'Lie To Kick It',
   "Fuck All Y'All",
   'Let Them Thangs Go',
   'Definition Of A Thug Nigga',
   'Ready 4 Whatever',
   'When I Get Free',
   'Hold On Be Strong',
   "I'm Losin It",
   'Fake Ass Bitches',
   'Do For Love',
   'Enemies With Me'],
  'All Eyez On Me': ['Ambitionz Az A Ridah',
   'All About U (ft. Nate Dogg, Snoop Dogg, Fatal, Yani Hadati)',
   'Skandalouz (ft. Nate Dogg)',
   'Got My Mind Made Up (ft. Dat Nigga Daz, Kurupt, Method Man, Redman)',
   'How Do You Want It (ft. KC, Jojo)',
   '2 Of Amerikaz Most Wanted (ft. Snoop Doggy Dog)',
   'No More Pain',
   'Heartz Of Men',
   'Life Goes On',
   "Only God Can Judge Me (ft. Rappin' 4-Tay)",
   'Tradin War Stories (ft. C-BO, CPO, Outlawz, The Storm)',


In [51]:
with open('full_song_dict_east_HUGE.json', 'w') as fp:
    json.dump(full_song_dict_east, fp)

In [39]:
with open('full_song_dict_west_HUGE.json', 'w') as fp:
    json.dump(full_song_dict_west, fp)